# Trial 2
something different, maybe. Hope this works! :')

In [1]:
import pandas as pd
from tqdm import tqdm
import pickle
import numpy as np
import re
import math

In [2]:
# with open("../df.pkl", "rb") as f:
#     df = pickle.load(f)
df = pd.read_excel('../data/merged_set.xlsx')

## Tools testing

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from yake import KeywordExtractor
# from keybert import KeyBERT
from transformers import pipeline

/mnt/Data Volume/Projects/Storion/Codebase/storion_aggregation_analysis/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


### YAKE (Looks good, but too many keywords)

In [6]:
kw_extractor = KeywordExtractor()

In [7]:
text = df.loc[df["event_id"] == "case_1"].iloc[0]["content"]

In [8]:
keywords = kw_extractor.extract_keywords(text)

In [9]:
keywords

[('allowing Elon Musk', np.float64(0.0012443619293702544)),
 ('Elon Musk America', np.float64(0.0016008832896744504)),
 ('Musk America PAC', np.float64(0.005631785765392376)),
 ('Elon Musk', np.float64(0.006004674381945689)),
 ('allowing Elon', np.float64(0.015793797497147603)),
 ('Musk America', np.float64(0.020240416352859676)),
 ('Pennsylvania judge', np.float64(0.02226199325158571)),
 ('Judge Angelo Foglietta', np.float64(0.03145404645119408)),
 ('America PAC', np.float64(0.035297120716982114)),
 ('Pleas Court Judge', np.float64(0.05132581560866452)),
 ('Court Judge Angelo', np.float64(0.05132581560866452)),
 ('Common Pleas Court', np.float64(0.05474920997337077)),
 ('contest through Nov.', np.float64(0.05721328937878896)),
 ('ruling on Monday.', np.float64(0.05721328937878896)),
 ('America PAC attorney', np.float64(0.06127688564125262)),
 ('Nov.', np.float64(0.06536876963257338)),
 ('Monday.', np.float64(0.06536876963257338)),
 ('Pennsylvania', np.float64(0.0772582259727435)),
 ('

In [10]:
kw_sum = 0
kw_sum = sum([keyword[1] for keyword in keywords])
kw_sum

np.float64(0.8351456024090593)

### Keybert (Useless)

In [9]:
model = KeyBERT(model="all-mpnet-base-v2")

In [10]:
keywords_kb = model.extract_keywords(text)
keywords_kb

[('contest', 0.3704),
 ('pac', 0.3642),
 ('election', 0.363),
 ('trump', 0.3499),
 ('musk', 0.3343)]

### NER_xlm-roberta-large-finetuned-conll03-english

In [5]:
fair_ner_pipe = pipeline("token-classification", model="FacebookAI/xlm-roberta-large-finetuned-conll03-english")

RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.

In [12]:
raw_entities = fair_ner_pipe(text)

In [13]:
text

'A Pennsylvania judge is allowing Elon Musk\'s America PAC to continue their $1 million a day contest through Nov. 5, according to a ruling on Monday.. . The ruling by Common Pleas Court Judge Angelo Foglietta comes after it was revealed during court proceedings that the contest\'s winners are not randomly selected.. . "The $1 million recipients are not chosen by chance," America PAC attorney Chris Gober said in court on Monday. "We know exactly who will be announced as the $1 million recipient today and tomorrow.". . NIKKI HALEY PENS SUPPORTIVE OP-ED IN FAVOR OF TRUMP AHEAD OF ELECTION DAY: \'EASY CALL\'. . CLICK HERE TO GET THE FOX NEWS APP. . The Associated Press contributed to this report.. . This is a breaking news situation. Check back with us for updates.'

In [14]:
raw_entities

[{'entity': 'I-LOC',
  'score': np.float32(0.99997306),
  'index': 2,
  'word': '▁Pennsylvania',
  'start': 2,
  'end': 14},
 {'entity': 'I-ORG',
  'score': np.float32(0.9167749),
  'index': 5,
  'word': '▁',
  'start': 24,
  'end': 25},
 {'entity': 'I-PER',
  'score': np.float32(0.9999901),
  'index': 7,
  'word': '▁El',
  'start': 33,
  'end': 35},
 {'entity': 'I-PER',
  'score': np.float32(0.99999344),
  'index': 8,
  'word': 'on',
  'start': 35,
  'end': 37},
 {'entity': 'I-PER',
  'score': np.float32(0.9999771),
  'index': 9,
  'word': '▁Musk',
  'start': 38,
  'end': 42},
 {'entity': 'I-ORG',
  'score': np.float32(0.9999908),
  'index': 12,
  'word': '▁America',
  'start': 45,
  'end': 52},
 {'entity': 'I-ORG',
  'score': np.float32(0.7667245),
  'index': 13,
  'word': '▁',
  'start': 53,
  'end': 54},
 {'entity': 'I-ORG',
  'score': np.float32(0.99995124),
  'index': 14,
  'word': 'PAC',
  'start': 53,
  'end': 56},
 {'entity': 'I-ORG',
  'score': np.float32(0.9999951),
  'index

In [6]:
def combine_same_entities(_text, _raw_entities):
    prev_segment = None
    entities = []
    for segment in _raw_entities:
        segment['word'] = segment['word'].replace('▁', ' ')
        original_entity_word = segment['word']
        entity_word = segment['word'].rstrip()
        segment['end'] = segment['end'] - (len(original_entity_word) - len(entity_word))
        if segment['word'].isspace():
            continue
        appended_to_prev_segment = False
        if prev_segment is not None and prev_segment['entity'] == segment['entity']:
            if prev_segment['end'] == segment['start']:
                entities[-1]['word'] += segment['word']
                appended_to_prev_segment = True
            elif _text[prev_segment['end']:segment['start']].isspace():
                entities[-1]['word'] += _text[prev_segment['end']:segment['start']] + segment['word']
                appended_to_prev_segment = True

            if appended_to_prev_segment:
                entities[-1]['end'] = segment['end']
                entities[-1]['score'] = (entities[-1]['score'] + segment['score'])/2

        if not appended_to_prev_segment:
            original_entity_word = entity_word
            entity_word = entity_word.lstrip()
            segment['start'] = segment['start'] + (len(original_entity_word) - len(entity_word))
            entities.append({
                'entity': segment['entity'],
                'word': entity_word,
                'score': segment['score'],
                'start': segment['start'],
                'end': segment['end']
            })
        entities[-1]['word'] = re.sub(r' +', ' ', entities[-1]['word']).strip()
        prev_segment = segment.copy()
    return entities

In [16]:
combine_same_entities(text, raw_entities)

[{'entity': 'I-LOC',
  'word': 'Pennsylvania',
  'score': np.float32(0.99997306),
  'start': 3,
  'end': 14},
 {'entity': 'I-PER',
  'word': 'Elon Musk',
  'score': np.float32(0.99998444),
  'start': 34,
  'end': 42},
 {'entity': 'I-ORG',
  'word': 'America PAC',
  'score': np.float32(0.99997103),
  'start': 46,
  'end': 56},
 {'entity': 'I-ORG',
  'word': 'Common Pleas Court',
  'score': np.float32(0.999995),
  'start': 167,
  'end': 184},
 {'entity': 'I-PER',
  'word': 'Angelo Foglietta',
  'score': np.float32(0.9999894),
  'start': 192,
  'end': 207},
 {'entity': 'I-ORG',
  'word': 'America PAC',
  'score': np.float32(0.9999592),
  'start': 373,
  'end': 383},
 {'entity': 'I-PER',
  'word': 'Chris Gober',
  'score': np.float32(0.99999356),
  'start': 394,
  'end': 404},
 {'entity': 'I-PER',
  'word': 'NIKKI HALEY',
  'score': np.float32(0.9999871),
  'start': 522,
  'end': 532},
 {'entity': 'I-PER',
  'word': 'TRUMP',
  'score': np.float32(0.9999835),
  'start': 568,
  'end': 572},


## Approach

In [7]:
from scripts.trial_2.text_filteration import ContentFilterer

In [8]:
content_filter = ContentFilterer()

In [9]:
selected_df = df.iloc[:200]
selected_df = selected_df._append(df.loc[df['event_id'] == 'case_1'])
selected_df = selected_df._append(df.loc[df['event_id'] == 'case_2'])
selected_df = selected_df._append(df.loc[df['event_id'] == 'case_3'])

In [10]:
selected_df.tail()

,id,title,author,publication_date,source,url,content,tags,images,event_id
197,a49cb3a4-6fb3-4bb0-b8b8-4631c85d7669,IIT Roorkee team finds superbug’s defense mech...,"['Suraj Maurya', 'Margin-Bottom Important', '....",NaT,The Press United | International News Analysis...,NaN,Facebook Twitter Reddit WhatsApp LinkedIn Pint...,"['Health', 'Education', 'India', 'Entertainmen...",https://thepressunited.com/wp-content/uploads/...,134
198,5747aab6-275a-4986-8d28-2a291f241b07,17p spice could stop 'superbugs' that can't be...,"['Andrew Nuttall', 'Image', 'Getty', 'Newsbyan...",NaT,Surrey Live,NaN,"Something went wrong, please try again later.\...",NaN,https://i2-prod.getsurrey.co.uk/news/health/ar...,134
199,3ac6af53-b337-4a2a-88c5-92fe3d45c0cb,IIT Roorkee team finds superbug’s defense mech...,"['Newsroom Network', 'Newsroom Odisha Network']",NaT,News Room Odisha,NaN,Roorkee: Scientists at the Indian Institute of...,NaN,https://www.newsroomodisha.com/wp-content/uplo...,134
76,eafa01c6-3078-49bf-913f-71697594fe1b,Heidi Klum unveils Halloween costume as ET wit...,"['Stephanie Giang-Paunon Larry Fink', 'Stephan...",NaT,FOX News,https://www.foxnews.com/entertainment/heidi-kl...,"The queen of Halloween, Heidi Klum, was ""out o...",['#HeidiHalloween'],['https://a57.foxnews.com/static.foxnews.com/f...,case_3
77,6e353ebb-749d-4520-be04-650305979a44,Heidi Klum Dresses as E.T. for Her Annual Hall...,NaN,NaT,TMZ,https://www.tmz.com/2024/11/01/heidi-klum-et-c...,Heidi Klum's Halloween bash is always a wild e...,NaN,['https://imagez.tmz.com/image/92/16by9/2024/1...,case_3


In [11]:
cleaned_contents = []

# For cleaning all the contents
# for idx, row in tqdm(df.iterrows(), desc="Filtering contents with CCF...", total=len(df)):
#     cleaned_content = content_filter.filter_text(row['title'], row['content'])
#     cleaned_contents.append(cleaned_content)

# For cleaning of selected sample set
for idx, row in tqdm(selected_df.iterrows(), desc="Filtering contents with CCF...", total=len(selected_df)):
    cleaned_content = content_filter.filter_text(row['title'], row['content'])
    cleaned_contents.append(cleaned_content)

Filtering contents with CCF...:  16%|█▌        | 32/202 [00:26<02:21,  1.21it/s]


KeyboardInterrupt: 

In [9]:
import pickle

with open("cleaned_contents.pkl", "wb") as f:
    pickle.dump(cleaned_contents, f)

In [12]:
import pickle

with open("cleaned_contents.pkl", "rb") as f:
    cleaned_contents = pickle.load(f)

In [13]:
# Sample of keyword extraction & NER
_sample_text = " ".join(cleaned_contents[19])
print(_sample_text)
sample_keywords = kw_extractor.extract_keywords(_sample_text)
print(sample_keywords)
sample_entities = combine_same_entities(_sample_text, fair_ner_pipe(_sample_text))
sample_entities

When will mail-in and absentee ballots be counted?  . One such hold-up is the work it takes to process and count mail-in ballots  . Because each state has different rules regarding the timeline for processing and counting mail-in ballots, many election officials cannot start tabulating these early ballots until Election Day or even after the polls close that night  Make sure you are aware of when your state starts processing and counting mail-in ballots  Local laws limit when election officials can process ballots  Absentee and mail-in ballots must be processed before they can be counted  . The process of handling mailed ballots varies by state, according to the National Conference of State Legislatures. Typically, it includes several steps: checking the ballot envelope, verifying that the signature on the return envelope matches the voter's signature on file, opening the envelope, and preparing the ballot for counting. In most states, officials cannot feed the ballot into the tabulato

[{'entity': 'I-MISC',
  'word': 'Election Day',
  'score': np.float32(0.99986774),
  'start': 314,
  'end': 325},
 {'entity': 'I-ORG',
  'word': 'National Conference of State Legislatures',
  'score': np.float32(0.99933076),
  'start': 672,
  'end': 712},
 {'entity': 'I-MISC',
  'word': 'Election Day',
  'score': np.float32(0.99974537),
  'start': 1028,
  'end': 1039},
 {'entity': 'I-LOC',
  'word': 'Virgin Island',
  'score': np.float32(0.9838215),
  'start': 1148,
  'end': 1160},
 {'entity': 'I-MISC',
  'word': 'Election Day',
  'score': np.float32(0.9997926),
  'start': 1235,
  'end': 1246},
 {'entity': 'I-MISC',
  'word': 'Election Day',
  'score': np.float32(0.99977684),
  'start': 1323,
  'end': 1334},
 {'entity': 'I-LOC',
  'word': 'Connecticut',
  'score': np.float32(0.9998577),
  'start': 1340,
  'end': 1350},
 {'entity': 'I-LOC',
  'word': 'Ohio',
  'score': np.float32(0.99997365),
  'start': 1356,
  'end': 1359},
 {'entity': 'I-LOC',
  'word': 'Alabama',
  'score': np.float3

In [14]:
def filter_non_entity_keywords(_keywords, _entities):
    entity_list = set([entity['word'] for entity in _entities])
    _keywords = [keyword for keyword in _keywords if keyword[0] not in entity_list]
    return _keywords

In [1]:
from difflib import SequenceMatcher
from collections import defaultdict

# Function to calculate similarity between two strings
def calculate_similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

# Function to create a similarity matrix for all words
def create_similarity_matrix(primary_word_list, secondary_word_list):
    height = len(primary_word_list)
    width = len(secondary_word_list)
    matrix = [[0] * width for _ in range(height)]

    for i, word1 in enumerate(primary_word_list):
        for j, word2 in enumerate(secondary_word_list):
            if i != j:  # Avoid self-comparison
                matrix[i][j] = calculate_similarity(word1, word2)
    return matrix

# Function to group words based on similarity matrix
def group_similar_words(word_list, top_values_threshold=None):
    similarity_matrix = np.array(create_similarity_matrix(word_list, word_list))
    # Dynamic thresholding 
    top_n_values = len(similarity_matrix[similarity_matrix>top_values_threshold]) if top_values_threshold is not None else similarity_matrix.shape[0]//1
    top_n_values = 1 if top_n_values==0 else top_n_values
    top_n_values *= -1
    top_n_similarities = np.partition(similarity_matrix.flatten(), top_n_values)[top_n_values:]
    threshold = np.mean(top_n_similarities)
    print(f"Threshold set to {threshold}")
    groups = []
    visited = set()

    for i, word in enumerate(word_list):
        if word not in visited:
            group = set()
            for j, similarity in enumerate(similarity_matrix[i]):
                if similarity >= threshold:
                    group.add(word_list[j])
            group.add(word)  # Include the current word
            groups.append(list(group))
            visited.update(group)  # Mark all words in this group as visited

    return groups

In [16]:
sample_kw_groups = group_similar_words([keyword[0] for keyword in sample_keywords], top_values_threshold=0.3)
sample_kw_groups

Threshold set to 0.4924780791305558


[['Election Day', 'election', 'election officials'],
 ['mail-in ballots',
  'absentee ballots',
  'early ballots',
  'ballots',
  'mail ballots'],
 ['Day', 'early'],
 ['start processing early',
  'mail-in ballots',
  'absentee ballots',
  'processing',
  'counting mail-in ballots',
  'early ballots',
  'mail ballots',
  'processing early ballots'],
 ['polls close'],
 ['mail-in ballots', 'mail-in'],
 ['absentee ballots', 'absentee'],
 ['states'],
 ['election', 'counting', 'counting mail-in ballots'],
 ['officials', 'election officials']]

In [17]:
SequenceMatcher(None, sample_kw_groups[0][1], sample_entities[2]['word']).ratio()

0.7

In [18]:
print(sample_kw_groups[0][1])
print(sample_entities[2]['word'])

election
Election Day


In [ ]:
def identify_same_entities(entities, threshold=0.55):
    entity_word_list = [entity['word'] for entity in entities]
    entity_similarities = np.array(create_similarity_matrix(entity_word_list, entity_word_list))
    entity_groups = []
    group_id = 0
    entity_group_ids = [None] * len(entities)
    is_grouped = [False] * len(entities)
    for i, entity in enumerate(entities):
        if not is_grouped[i]:
            group = [entity]
            is_grouped[i] = True
            entity_group_ids[i] = group_id
            shorlisted_similarities = (entity_similarities[i, :] > threshold)
            for j, is_shorlisted in enumerate(shorlisted_similarities):
                if not is_shorlisted or is_grouped[j]:
                    continue
                if entity['entity'] == entities[j]['entity']:
                    group.append(entities[j])
                    is_grouped[j] = True
                    entity_group_ids[j] = group_id
            entity_groups.append(group)
            group_id += 1
    return entity_groups, entity_group_ids

In [20]:
sample_entity_groups, sample_entity_group_ids = identify_same_entities(entities=sample_entities, threshold=0.55)
sample_entity_groups

[[{'entity': 'I-MISC',
   'word': 'Election Day',
   'score': np.float32(0.99986774),
   'start': 314,
   'end': 325},
  {'entity': 'I-MISC',
   'word': 'Election Day',
   'score': np.float32(0.99974537),
   'start': 1028,
   'end': 1039},
  {'entity': 'I-MISC',
   'word': 'Election Day',
   'score': np.float32(0.9997926),
   'start': 1235,
   'end': 1246},
  {'entity': 'I-MISC',
   'word': 'Election Day',
   'score': np.float32(0.99977684),
   'start': 1323,
   'end': 1334},
  {'entity': 'I-MISC',
   'word': 'Election Day',
   'score': np.float32(0.999661),
   'start': 1706,
   'end': 1717},
  {'entity': 'I-MISC',
   'word': 'Election Day',
   'score': np.float32(0.9997833),
   'start': 1969,
   'end': 1980},
  {'entity': 'I-MISC',
   'word': 'Election Day',
   'score': np.float32(0.99972713),
   'start': 2274,
   'end': 2285}],
 [{'entity': 'I-ORG',
   'word': 'National Conference of State Legislatures',
   'score': np.float32(0.99933076),
   'start': 672,
   'end': 712}],
 [{'entity

In [ ]:
from dataclasses import dataclass

@dataclass
class Entity:
    name: str
    entity_type: str
    score: np.float64

In [21]:
def link_entity_relevant_keyword_groups(keyword_groups, entity_groups, entity_group_ids):
  entity_keyword_group_ids = dict() # Initialize dictionary to store keyword group ids for each entity group
  max_keyword_group_length = max([len(keyword_group) for keyword_group in keyword_groups]) # Get maximum number of keywords in a group
  keyword_lengths = np.array([[len(keyword) for keyword in keyword_group] + ([-1]*(max_keyword_group_length-len(keyword_group))) for keyword_group in keyword_groups]) # Get lengths of all keywords in each group
  for group_id, entity_group in enumerate(entity_groups): # Iterate over each entity group
    entity_keyword_group_ids[group_id] = [] # Initialize list to store keyword group ids for the entity group
    entity_words = [entity['word'] for entity in entity_group] # Get all words in the entity group
    entity_word_lengths = [len(entity['word']) for entity in entity_group] # Get lengths of all words in the entity group
    compared_entity_word = entity_words[np.argmax(entity_word_lengths)] # Get the longest word in the entity group
    compared_entity_word_length = len(compared_entity_word) # Get the length of the longest word in the entity group
    # Threshold need to be non-linearly inversed propotional (the more length of the word, the less propotion of the length of the word as threshold, the threshold = length x propotion, the propotion should be between 0 and 1)
    default_proportion = 0.7 # For 3 words
    acceptable_keyword_length_threshold = compared_entity_word_length * (default_proportion / math.log(compared_entity_word_length + 1, 10)) # Non-linearly inversed propotional threshold
    acceptable_keyword_length_threshold = 1 if acceptable_keyword_length_threshold < 1 else acceptable_keyword_length_threshold # Minimum threshold to 1 (in case of very short words)
    acceptable_keyword_length = [compared_entity_word_length - acceptable_keyword_length_threshold, compared_entity_word_length + acceptable_keyword_length_threshold] # Acceptable length range for keywords with the threshold
    shortlisted_keyword_groups = [any(group_checks) for group_checks in np.bitwise_and(keyword_lengths >= acceptable_keyword_length[0], keyword_lengths <= acceptable_keyword_length[1])] # Shortlist groups with atleast one keyword in the acceptable length range
    for i, is_shortlisted in enumerate(shortlisted_keyword_groups): # Iterate over shortlisted keyword groups
        # Exit if the keyword group is not shortlisted
        if not is_shortlisted:
            continue
        # Check if the entity group is relevant to the keyword group
        similarity_scores = [SequenceMatcher(None, entity_word, keyword).ratio() for entity_word in entity_words for keyword in keyword_groups[i]]
        if max(similarity_scores) == 1 or np.mean(similarity_scores) > 0.8:
            entity_keyword_group_ids[group_id].append(i)
    entity_keyword_group_ids[group_id] = None if len(entity_keyword_group_ids[group_id]) == 0 else entity_keyword_group_ids[group_id] # Set to None if no keyword group is relevant to the entity group
  return entity_keyword_group_ids

In [22]:
entity_keyword_group_links = link_entity_relevant_keyword_groups(sample_kw_groups, sample_entity_groups, sample_entity_group_ids)
sample_entity_group_word_list = [[entity['word'] for entity in entity_group] for entity_group in sample_entity_groups]
{ max(sample_entity_group_word_list[entity_group_id], key=len): None if relevent_kw_group_ids is None else [sample_kw_groups[kw_group_id] for kw_group_id in relevent_kw_group_ids] for entity_group_id, relevent_kw_group_ids in entity_keyword_group_links.items()}

{'Election Day': [['Election Day', 'election', 'election officials']],
 'National Conference of State Legislatures': None,
 'Virgin Island': None,
 'Connecticut': None,
 'Ohio': None,
 'Alabama': None,
 'Mississippi': None,
 'New Hampshire': None,
 'Pennsylvania': None,
 'South Dakota': None,
 'West Virginia': None,
 'Wisconsin': None,
 'Washington, D.C.': None}

In [23]:
sample_entity_group_word_list

[['Election Day',
  'Election Day',
  'Election Day',
  'Election Day',
  'Election Day',
  'Election Day',
  'Election Day'],
 ['National Conference of State Legislatures'],
 ['Virgin Island'],
 ['Connecticut'],
 ['Ohio'],
 ['Alabama'],
 ['Mississippi'],
 ['New Hampshire'],
 ['Pennsylvania'],
 ['South Dakota'],
 ['West Virginia'],
 ['Wisconsin'],
 ['Washington, D.C.']]

In [24]:
max(sample_entity_group_word_list[0], key=len)

'Election Day'

In [25]:
sample_kw_groups

[['Election Day', 'election', 'election officials'],
 ['mail-in ballots',
  'absentee ballots',
  'early ballots',
  'ballots',
  'mail ballots'],
 ['Day', 'early'],
 ['start processing early',
  'mail-in ballots',
  'absentee ballots',
  'processing',
  'counting mail-in ballots',
  'early ballots',
  'mail ballots',
  'processing early ballots'],
 ['polls close'],
 ['mail-in ballots', 'mail-in'],
 ['absentee ballots', 'absentee'],
 ['states'],
 ['election', 'counting', 'counting mail-in ballots'],
 ['officials', 'election officials']]

In [26]:
# For the moment flatten the ner and keyword groups by length
proceeding_entity_list = [max(entity_group, key=len) for entity_group in sample_entity_group_word_list]
proceeding_keyword_list = [max(keyword_group, key=len) for keyword_group in sample_kw_groups]
proceeding_keyword_list = list(set(proceeding_keyword_list))

In [27]:
proceeding_keyword_list

['states',
 'mail-in ballots',
 'election officials',
 'early',
 'absentee ballots',
 'counting mail-in ballots',
 'polls close']

### Article Grouping

In [7]:
import spacy

In [8]:
!python -m spacy download en_core_web_md

     ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.5/33.5 MB ? eta -:--:--
ERROR: Operation cancelled by user
^C


In [ ]:
# Load the spacy model
spacy_model = spacy.load('en_core_web_sm')

In [ ]:
def get_entity_similarity_matrix(primary_entity_list, secondary_entity_list):
  name_similarity_matrix = np.array(create_similarity_matrix(
    [entity.word for entity in primary_entity_list], 
    [entity.word for entity in secondary_entity_list]
    ))
  type_match_matrix = np.array([[1 if entity.entity_type == entity2.entity_type else 0 for entity2 in secondary_entity_list] for entity in primary_entity_list])
  assert name_similarity_matrix.shape == type_match_matrix.shape # Ensure the shape of both matrices are the same
  final_score_matrix = name_similarity_matrix * type_match_matrix
  return final_score_matrix

In [ ]:
entity_weights = {
  "Person": 0.4,
  "Location": 0.5,
  "Organization": 0.3,
  "Miscellaneous": 0.2
}

class ArticleGroup:
  identifier: int
  __internal_entities: list[str]
  __internal_keywords: list[str]
  __group_article_ids: list[str]
  __total_entity_scorable: float

  def __init__(self, identifier: int = -1):
    self.identifier = identifier
    self.__internal_entities = []
    self.__total_entity_scorable = 0
    self.__internal_keywords = []

  def get_group_article_ids(self):
    return self.__group_article_ids
  
  def get_group_keywords(self):
    return self.__internal_keywords
  
  def get_group_entities(self):
    return self.__internal_entities
  
  def add_article_id(self, article_id: str, entities: list[str], keywords: list[str]):
    global entity_weights
    self.__group_article_ids.append(article_id)
    ## Total entity scorable calculation
    self.__total_entity_scorable = sum([entity_weights[entity.entity_type] for entity in entities])
    pass

  def get_group_relevance_score(self, entities: list[str], keywords: list[str], entity_match_threshold: float = 0.8, keyword_match_threshold: float = 0.8) -> float:
    global entity_weights
    # Check if the entities are relevant to the group (make sure the article is about the same subject and people) (40% weight)
    ## Get entity similarity score matrix
    entity_similarity_matrix = get_entity_similarity_matrix(self.__internal_entities, entities)
    ## Create entity type matrix
    no_of_article_entities = len(entities)
    entity_type_matrix = []
    for i, entity in enumerate(self.__internal_entities):
      max_index = np.argmax(entity_similarity_matrix[i])
      if entity_similarity_matrix[i, max_index] < entity_match_threshold:
        entity_type_matrix.append([None] * no_of_article_entities)
      else:
        entity_type_matrix.append([None] * max_index + [entities[max_index].entity_type] + [None] * (no_of_article_entities - max_index - 1))
    entity_type_matrix = np.array(entity_type_matrix)
    # entity_type_matrix = np.array([[entity.entity_type if entity_similarity_matrix[i][j] > entity_match_threshold else None for j in range(no_of_article_entities)] for i, entity in enumerate(self.__internal_entities)])
    ## Calculate the entity weight matrix
    entity_weight_matrix = np.array([[entity_weights[entity_type] if entity_type is not None else 0 for entity_type in entity_type_matrix_row] for entity_type_matrix_row in entity_type_matrix])
    ## Calculate the entity score
    entity_score = np.sum(entity_weight_matrix)
    ## Normalize the entity score to the 40% weight
    normalized_entity_score = (entity_score / self.__total_entity_scorable) * 0.4
    if normalized_entity_score == 0: return 0.0
    # Check if the keywords are relevant to the group (make sure the article is talking about the same event within the subject) (60% weight)
    ## Get keyword similarity matrix
    kw_similarity_matrix = np.array(create_similarity_matrix(self.__internal_keywords, keywords))
    ## Calculate the keyword score
    normalized_keyword_score = (np.sum(kw_similarity_matrix) / len(keywords)) * 0.6 if keywords else 0.0
    # Return the total score
    return normalized_entity_score + normalized_keyword_score